In [1]:
import pandas as pd
import numpy as np
import tqdm
import tpot
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import dask_ml.model_selection

In [2]:
train = pd.read_csv('train_one_hot2.csv')
test = pd.read_csv('test_one_hot2.csv')

In [3]:
X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]

In [4]:
X_train.shape

(116368, 1033)

In [5]:
y_train.shape
y_train.head()

0    0
1    0
2    1
3    1
4    0
Name: RainTomorrow, dtype: int64

In [6]:
X_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

In [7]:
y_test.dtypes

dtype('int64')

In [8]:
y_train=y_train.values.ravel()
y_test=y_test.values.ravel()

In [9]:
from dask.distributed import Client, Variable


In [15]:
client=Client()
client

F:\Anaconda3\envs\cudapy3\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58153 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:58153/status,
Dashboard: http://127.0.0.1:58153/status,Workers: 4
Total threads: 12,Total memory: 15.85 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58154,Workers: 4
Dashboard: http://127.0.0.1:58153/status,Total threads: 12
Started: Just now,Total memory: 15.85 GiB
Comm: tcp://127.0.0.1:58184,Total threads: 3
Dashboard: http://127.0.0.1:58185/status,Memory: 3.96 GiB
Nanny: tcp://127.0.0.1:58160,


In [11]:
digits = load_digits()

In [21]:
TP = TPOTClassifier(generations=3,population_size=10,cv=5,n_jobs=-1,use_dask=False)

In [22]:
TP.fit(X_train,y_train)

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x000002B15EE1BD30>>, <Task finished name='Task-190618' coro=<Cluster._sync_cluster_info() done, defined at F:\Anaconda3\envs\cudapy3\lib\site-packages\distributed\deploy\cluster.py:104> exception=OSError('Timed out trying to connect to tcp://127.0.0.1:58154 after 30 s')>)
Traceback (most recent call last):
  File "F:\Anaconda3\envs\cudapy3\lib\site-packages\distributed\comm\core.py", line 284, in connect
    comm = await asyncio.wait_for(
  File "F:\Anaconda3\envs\cudapy3\lib\asyncio\tasks.py", line 501, in wait_for
    raise exceptions.TimeoutError()
asyncio.exceptions.TimeoutError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "F:\Anaconda3\envs\cudapy3\lib\site-packages\tornado\ioloop.py", line 741, in _run_callback
    ret = callback()
  File "F:\Anaconda3\env

TPOTClassifier(generations=3, n_jobs=-1, population_size=10)

In [23]:
TP.export('best_model3.py')

In [30]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC

In [31]:
exported_pipeline = make_pipeline(
    MinMaxScaler(),
    LinearSVC(C=20.0, dual=False, loss="squared_hinge", penalty="l2", tol=1e-05)
)

exported_pipeline.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('linearsvc', LinearSVC(C=20.0, dual=False, tol=1e-05))])

In [33]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [34]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [35]:
scores = cross_val_score(exported_pipeline, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [36]:
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Accuracy: 0.848 (0.004)


In [37]:
exported_pipeline.score(X_test,y_test)

0.8468307438471058

In [38]:
client.close()